# Multilayer Perceptrons

In [ ]:
from collections import Counter
from dataclasses import dataclass
from typing import Optional

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from nnfs.activations import Activation
from nnfs.layers import ActivationLayer, BatchNorm, Dense, Dropout
from nnfs.losses import LossFunction
from nnfs.models.neural_network import NerualNetwork
from nnfs.optimizers import Optimizer
from nnfs.utils import Preprocessing

np.set_printoptions(precision=4)

# configuration
TEST_RATIO = 0.15
VALIDATION_RATIO = 0.15


@dataclass
class HyperParams:
    wine_type: Optional[str] = None
    subset: Optional[str] = None
    random_seed: int = 42
    test_ratio: float = 0.15
    validation_ratio: float = 0.15
    learning_rate: float = 1e-4
    batch_size: int = 8
    epochs: int = int(2e2)

    def __post_init__(self):
        self.training_ratio = 1 - (self.test_ratio + self.validation_ratio)
        self.test_to_val_ratio = self.validation_ratio / (
            self.test_ratio + self.validation_ratio
        )


params = HyperParams("both", None)

In [ ]:
np.random.seed(HyperParams.random_seed)
white_wine_csv = "../data/raw/winequality-white.csv"
red_wine_csv = "../data/raw/winequality-red.csv"

white_wine = pd.read_csv(white_wine_csv, delimiter=";")
red_wine = pd.read_csv(red_wine_csv, delimiter=";")


match params.subset:

    case "extreme":
        filter_vals = [1, 2, 3, 4, 8, 9, 10]
        white_wine = white_wine[white_wine["quality"].isin(filter_vals)]
        red_wine = red_wine[red_wine["quality"].isin(filter_vals)]

    case default:
        pass

In [ ]:
white_wine_raw = white_wine.to_numpy()
red_wine_raw = red_wine.to_numpy()

match params.wine_type:

    case "red":
        wines_raw = red_wine_raw
    case "white":
        wines_raw = white_wine_raw
    case default:
        wines_raw = np.concatenate((white_wine_raw, red_wine_raw))

wines_raw = white_wine_raw

wines_raw, wines_mean, wines_std = Preprocessing.standard_scale(wines_raw)

test, train = Preprocessing.train_test_split(
    wines_raw, 1 - params.training_ratio, shuffle=True
)
test, validation = Preprocessing.train_test_split(test, params.test_to_val_ratio)

x_train, y_train = Preprocessing.xy_split(train)
x_test, y_test = Preprocessing.xy_split(test)
x_val, y_val = Preprocessing.xy_split(validation)

x_train.shape, x_test.shape, x_val.shape

In [ ]:
optim = Optimizer.adam(learning_rate=params.learning_rate)

model = NerualNetwork(
    optimizer=optim,
    loss_fn=LossFunction.squared_error(),
    validation_data=(x_val, y_val),
)

prev_shape = model.add(Dense(33, (params.batch_size, x_train.shape[1]), optim))

prev_shape = model.add(ActivationLayer(prev_shape, Activation.leakyRelu()))

prev_shape = model.add(Dense(22, prev_shape, optim))

prev_shape = model.add(ActivationLayer(prev_shape, Activation.leakyRelu()))

prev_shape = model.add(Dropout(prev_shape, p_drop=0.3))

prev_shape = model.add(Dense(11, prev_shape, optim))

prev_shape = model.add(ActivationLayer(prev_shape, Activation.leakyRelu()))

prev_shape = model.add(Dropout(prev_shape, p_drop=0.3))

output_shape = model.add(Dense(1, prev_shape, optim))

output_shape

In [ ]:
errors = model.fit(
    x_train, y_train, params.epochs, batch_size=params.batch_size, live_update=True
)

In [ ]:
loss, accuracy, y_pred = model.predict(x_test, y_test)

y_pred = (y_pred * wines_std[0, -1]) + wines_mean[0, -1]
y_test_ = (y_test * wines_std[0, -1]) + wines_mean[0, -1]

print(f"Loss: {loss}, R2 Score: {accuracy}")

print(f"Predictions: {np.round(y_pred[10:20].T)}\nTrue Values: {y_test_[10:20].T}")


print("Distribution of True Values", Counter(np.round(y_test_).flatten().tolist()))
print("Distribution of Predictions", Counter(np.round(y_pred).flatten().tolist()))